In [2]:
# リスト5.2.1
# Word2Vec 学習用テキストの取得と整形（青空文庫：夏目漱石『三四郎』）
# 目的:
#   - 後段の形態素解析・Word2Vec 学習に使えるよう、一次テキストをダウンロード→解凍→本文抽出→前処理する。
# ポイント:
#   - 青空文庫の本文はヘッダ/フッタやルビ・注記を含むため、正規表現で除去してクリーンなコーパスを得る。
#   - 文字コードは Shift_JIS（sjis）で公開されることが多いので、明示的に指定して開く。
#   - 最終的に先頭/末尾を一部表示して、整形結果を目視確認する。

# =========================
# 1) zipファイルのダウンロード
# =========================
# 対象は夏目漱石『三四郎』のルビ付きテキスト（_ruby_ を含むファイル）
url = "https://www.aozora.gr.jp/cards/000148/files/794_ruby_4237.zip"
zip = "794_ruby_4237.zip"  # ※組込み関数名 'zip' と同名だが、ここでは変数として使用（実務では別名推奨）

import urllib.request

# 指定URLからローカルへダウンロード（戻り値は保存先パス, ヘッダ情報のタプル）
urllib.request.urlretrieve(url, zip)

# =========================
# 2) ダウンロードしたzipを解凍し、本文テキストを読み込み
# =========================
import zipfile

with zipfile.ZipFile(zip, "r") as myzip:
    # アーカイブ内の全ファイルをカレントディレクトリへ展開
    myzip.extractall()
    # 解凍後に含まれるファイル一覧を走査し、テキスト本文を読み込む
    # ルビ付きテキスト（.txt）が1つだけ入っている想定
    for myfile in myzip.infolist():
        filename = myfile.filename  # 例: 'sanshirou.txt' のようなファイル名
        # 青空文庫のテキストは多くが Shift_JIS（ここでは 'sjis' 指定）で配布されている
        # OS/環境差異で 'cp932' が必要になることもあるが、まずは 'sjis' で試す
        with open(filename, encoding="sjis") as file:
            text = file.read()

# =========================
# 3) 青空文庫特有の前処理（本文抽出・ノイズ除去）
# =========================
import re

# 青空文庫のメタ情報は '-----' による区切りで囲まれることが多い。
# re.split('\-{5,}', text) で "-----" 以上の連続ハイフンを区切りに分割し、
# 先頭から [0]=ヘッダ前半, [1]=ヘッダ, [2]=本文… のことが多いため [2] を採用。
# （作品により構造が微妙に異なる可能性がある点に注意）
text = re.split("\-{5,}", text)[2]

# フッタ（底本情報以降）を除去。
# '底本：' 以降は出典や注記なので学習には不要。
text = re.split("底本：", text)[0]

# 青空文庫の縦書き由来の区切り記号 '|' を削除（行内のルビ位置などのために使われる）
text = text.replace("|", "")

# ルビ（《…》で囲まれた読み仮名）を削除。学習用には表記ゆれの原因になるため除去。
text = re.sub("《.+?》", "", text)

# 入力注（［＃…］で囲まれた編集注記）を削除。こちらも本文外情報のため学習から除外。
text = re.sub("［＃.+?］", "", text)

# 連続する空行を1つに正規化（文区切りの過剰なノイズを抑える）
text = re.sub("\n\n", "\n", text)

# Windows系改行に混在しがちな '\r' を除去し、行末記号を '\n' に統一
text = re.sub("\r", "", text)

# =========================
# 4) 整形結果の確認（サンプル表示）
# =========================
# 冒頭100文字を表示して、ヘッダが除去され本文が先頭から始まっているかを目視確認
print(text[:100])

# 見やすさのための空行
print()
print()

# 末尾100文字を表示して、フッタが除去されているかを目視確認
print(text[-100:])

# 以上で、Word2Vec など分散表現学習のコーパスとして利用可能なプレーンテキストが得られる。
# 後続ステップ例：
#   - 形態素解析で分かち書き（名詞・動詞原形など抽出）
#   - トークン列を Gensim の Word2Vec に投入し学習
#   - 学習済みモデルで類似語検索/アナロジー推論を実施


一
　うとうととして目がさめると女はいつのまにか、隣のじいさんと話を始めている。このじいさんはたしかに前の前の駅から乗ったいなか者である。発車まぎわに頓狂な声を出して駆け込んで来て、いきなり肌をぬい


評に取りかかる。与次郎だけが三四郎のそばへ来た。
「どうだ森の女は」
「森の女という題が悪い」
「じゃ、なんとすればよいんだ」
　三四郎はなんとも答えなかった。ただ口の中で迷羊、迷羊と繰り返した。




In [3]:
# リスト5.2.2
# Word2Vec 学習用データの前処理
# 目的：
#   - 後段の Word2Vec 学習で入力する「文ごとのトークン列（list[list[str]]）」を作る。
#   - Janome で形態素解析し、名詞・動詞・形容詞の「原形（base_form）」のみを抽出する。
# 背景：
#   - Word2Vec は連続語彙モデル（CBOW/Skip-gram）により、隣接語の共起を手掛かりにベクトルを学習する。
#   - 品詞を絞ることでノイズ（助詞・記号など）を減らし、学習の安定性・計算効率を高める。
#   - 動詞・形容詞は活用形が多く、そのままだと語彙がスパース化するため「原形化」して語彙を統合するのが定石。
# 注意：
#   - Janome の Token.base_form は未知語の場合 '*' を返すことがある（必要なら surface へのフォールバックを検討）。
#   - 文分割を単純に '。' で行うと、引用・箇条書き・記号ゆれで分割精度が落ちる。厳密には文区切り器を使う。
#   - 大規模コーパスでは解析コストが高いので、事前のキャッシュや並列化、バッチ処理を検討する。

# Janomeのロード
from janome.tokenizer import Tokenizer

# Tokenizer は内部で辞書をロードするため、毎回生成せずに再利用するのが高速
t = Tokenizer()


# テキストを引数として、形態素解析の結果から
#   - 名詞・動詞・形容詞のみを対象
#   - 原形（base_form）を取り出す
# を行い、単語（文字列）リストを返す関数
def extract_words(text):
    # Tokenizer.tokenize は形態素ごとの Token オブジェクト（surface, base_form, part_of_speech 等を持つ）を返す
    tokens = t.tokenize(text)
    # part_of_speech は「品詞,品詞細分類1,品詞細分類2,品詞細分類3」のカンマ区切り文字列
    # 先頭要素（大分類）が '名詞'・'動詞'・'形容詞' のものだけを残す
    # base_form（原形）を返すことで、活用差による語彙の分断を回避して学習を安定させる
    return [
        token.base_form
        for token in tokens
        if token.part_of_speech.split(",")[0] in ["名詞", "動詞", "形容詞"]
    ]
    # 参考：
    #   - 名詞は原形＝表層形であることが多いが、固有名詞・未知語の扱いに注意
    #   - 動詞・形容詞は原形化の効果が大きい（例：「行く」「行った」「行き」→「行く」に正規化）


#  関数テスト（任意）：
# ret = extract_words('三四郎は京都でちょっと用があって降りたついでに。')
# for word in ret:
#     print(word)

# 文単位で学習させるため、まず全文を "。" で分割して文配列にする
# 重要：
#   - 末尾が "。" で終わらない場合、最後の要素は空でない短文になる。
#   - 「。」以外の終止記号（! ?、全角/半角の混在）や見出し行は別途正規化すると良い。
sentences = text.split("。")

# 各文を extract_words でトークン化し、list[list[str]] 形式に変換する
# 規模によっては数分～数十分かかるため、進捗表示（tqdm）や分割処理の導入が有効
word_list = [extract_words(sentence) for sentence in sentences]

# 結果の一部を確認（サンプル）：
# - 先頭2文分のトークン列を出力し、想定どおり「名詞・動詞・形容詞の原形」のみになっているかを確認する
# - インデックスエラー回避のため、最低限の要素数チェックを行ってもよい
print(word_list[0])
print(word_list[1])

# これで、Gensim の Word2Vec にそのまま投入できる学習用データ（コーパス）が整った。
# 次段例：
#   from gensim.models import Word2Vec
#   model = Word2Vec(sentences=word_list, vector_size=200, window=5, min_count=5, sg=1, workers=4)
#   model.wv.most_similar('学生')  # 類似語の確認 等

['一', 'する', '目', 'さめる', '女', '隣', 'じいさん', '話', '始める', 'いる']
['じいさん', '前', '前', '駅', '乗る', 'いなか者']


In [5]:
# ✅ エラー原因と修正方針
# - 例外: TypeError: Word2Vec.__init__() got an unexpected keyword argument 'size'
# - 原因: gensim 4.x では引数名が変更
#     * size   → vector_size
#     * iter   → epochs
#   （min_count, window, sg, workers などは継続利用可）
#
# 本スニペットでは gensim のバージョンを動的に判定し、
# 3.x/4.x の両方に対応して学習を実行する。
# さらに日本語コーパス（小規模前提）での典型的なハイパーパラメータの
# 理由付けをコメントに記述する。

import os
import gensim
from gensim.models import word2vec

# --- 入力前提 ---
# 前段で用意した文ごとのトークン列リスト:
#   word_list = [["三四郎","京都","用",...], ["彼","大学",...], ...]
# ※ 形態素解析＋原形化＋品詞フィルタを済ませておくと学習が安定する。

# --- ハイパーパラメータ選定の理屈 ---
# vector_size/size = 100:
#   小規模・中規模コーパスでのバランスが良い表現次元。
# window = 5:
#   文脈窓。小さめは構文類似（機能語の影響）、大きめは意味/トピック寄り。
# min_count = 5:
#   低頻度語の統計は不安定 → 語彙から除外して汎化を促す（必要に応じて 3 に下げる）。
# epochs/iter = 100:
#   小規模コーパスで統計を安定化させるため多めに回す。
# sg = 0 (CBOW):
#   既定。高速・安定。希少語を重視したいときは sg=1（Skip-gram）に変更。
# negative（省略時=5 前後）/sample/hs:
#   SGNS（負例サンプリング）のシフト付きPMI近似の観点で調整余地あり。


# --- バージョン判定（3.x と 4.x の引数差分に対応） ---
def _is_gensim4():
    try:
        # 例: '4.3.2' → (4,3,2)
        v = tuple(int(x) for x in gensim.__version__.split(".")[:3])
        return v >= (4, 0, 0)
    except Exception:
        # 取得失敗時は 4 系と仮定しない（保守的に 3.x とみなす）
        return False


# --- 学習本体 ---
if _is_gensim4():
    # gensim 4.x: vector_size / epochs を使用
    model = word2vec.Word2Vec(
        sentences=word_list,  # 4.x は引数名 sentences を推奨
        vector_size=100,  # ← size の代替
        window=5,
        min_count=5,
        sg=0,  # 0: CBOW, 1: Skip-gram
        workers=os.cpu_count() or 1,
        epochs=100,  # ← iter の代替
        seed=42,  # 再現性（完全固定は困難）
        # negative=5, sample=1e-3, hs=0 など必要に応じて追加
    )
else:
    # gensim 3.x: size / iter を使用
    model = word2vec.Word2Vec(
        word_list,  # 3.x は位置引数でも可
        size=100,  # ← 3.x の引数名
        window=5,
        min_count=5,
        sg=0,
        workers=os.cpu_count() or 1,
        iter=100,  # ← 3.x の引数名
        seed=42,
    )

# --- 学習後の簡易検証（語彙に無ければ握りつぶす） ---
# * コーパス由来の代表語を選ぶ（例: 「三四郎」「東京」など）
# * min_count の影響で語彙外になることがあるため try/except で保護
probe_words = ["三四郎", "東京", "大学"]
for w in probe_words:
    try:
        print("most_similar({}):".format(w), model.wv.most_similar(w, topn=10))
        break
    except KeyError:
        continue

# --- トラブルシューティング ---
# ・再現性が低い/近傍が毎回変わる:
#     → seed を固定、コーパスの順序を固定、workers を 1 に（ただし速度低下）
# ・意味類似が弱い/ノイズが多い:
#     → 前処理の表記統一（NFKC 正規化/半角全角/句読点除去）、
#       window と sample を調整、ユーザー辞書で複合語一語化
# ・希少だが重要な語の近傍を良くしたい:
#     → sg=1（Skip-gram）に変更、min_count を下げる、epochs を増やす

most_similar(三四郎): [('野々宮', 0.319937139749527), ('いる', 0.3083000183105469), ('膳', 0.2863658368587494), ('明瞭', 0.2770010828971863), ('する', 0.2730286121368408), ('の', 0.2692168653011322), ('一間', 0.26337337493896484), ('ランプ', 0.23961107432842255), ('下女', 0.2384023368358612), ('机', 0.2369156777858734)]
